In [36]:
import os
from lab_ia_slurm import gpu_jobs_submitter

# project root path
home = os.getenv("HOME")
root = os.path.join(home,"miti_laiking/code/semantic_similarity")

## train_cross

In [37]:
# Local Paths
run_path = f"{root}/fine-tuning/train_cross.py "
dataset_path = f"{root}/data/ak-outcome-sim"
out_dir_template = f"{root}/fine-tuning/out/{{exp_name}}"
# Different experiments Runs 
seed_nb = 1
models = [
    #"microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract",
    #"microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext",
    "michiyasunaga/BioLinkBERT-base",
]

In [38]:
# Torchrun (distributed training) Arguments
base_cmd = f"python {run_path} "
base_cmd += f"{dataset_path} "

cmds = []
exp_names = []

for model_path in models :
    for seed in range(seed_nb):
        cmd = base_cmd
        # Model
        cmd += f"{model_path} "
        # Experience name for output directory
        exp_name = f"{model_path.split('/')[-1]}_cross-encoder"
        exp_name += f"_seed{seed}"
        out_dir = out_dir_template.format(exp_name=exp_name)
        # Output dirs
        cmd += f"{out_dir} " # model_save_path
        cmd += f"{out_dir} " # test_save_path
        # Seed
        cmd += f"--seed {seed} "
        # fill lists
        if os.path.exists(os.path.join(out_dir,"predict_results.json")):
            print(exp_name, "already finished")
            continue
        cmds.append(cmd)
        exp_names.append(exp_name)

# Display experiences and chosen debug
for i,e in enumerate(exp_names):
    print(i,":",e)
print(cmds[0])

0 : BioLinkBERT-base_cross-encoder_seed0
python /mnt/beegfs/home/laiking/miti_laiking/code/semantic_similarity/fine-tuning/train_cross.py  /mnt/beegfs/home/laiking/miti_laiking/code/semantic_similarity/data/ak-outcome-sim michiyasunaga/BioLinkBERT-base /mnt/beegfs/home/laiking/miti_laiking/code/semantic_similarity/fine-tuning/out/BioLinkBERT-base_cross-encoder_seed0 /mnt/beegfs/home/laiking/miti_laiking/code/semantic_similarity/fine-tuning/out/BioLinkBERT-base_cross-encoder_seed0 --seed 0 


In [39]:
counter = 0
script_addon = """find . -type f -exec chown :miti_laiking {} +"""
for cmd,exp_name in zip(cmds, exp_names) :   
    if counter > 10: break
    # send job
    job_ids = gpu_jobs_submitter(
        cmd,
        exp_name = exp_name,
        time_max="24:00:00",
        script_addon=script_addon
    )
    counter+= 1

Submitted batch job 744930
1 GPUs / 4 cpus per task


In [41]:
!squeue -u laiking

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            744930       all BioLinkB  laiking  R      36:56      1 n1


In [4]:
!scancel -u laiking

In [35]:
# delete slurm files
!rm -rf slurm/*.slurm
!rm -rf **/.ipynb_checkpoints

## train_cross_hf